In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.4 MB/s eta 0:00:00


In [2]:
# Import library
from transformers import BertTokenizer
import tensorflow as tf
import pandas as pd
import transformers
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir("/content/drive/My Drive/")

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [7]:
# Preprocess the text data
X = df['review'].values
y = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

In [8]:
# Tokenize the input sentences and create attention masks
input_ids = []
attention_masks = []
for sentence in X:
    encoded_dict = tokenizer.encode_plus(
        sentence,
        add_special_tokens = True,
        max_length = 128,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        return_tensors = 'tf'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

In [9]:
# Convert the lists into TensorFlow tensors
input_ids = tf.concat(input_ids, axis=0)
attention_masks = tf.concat(attention_masks, axis=0)
y = tf.convert_to_tensor(y)
input_ids = tf.convert_to_tensor(input_ids)


In [10]:
# Split the dataset into training and testing sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids.numpy(), y.numpy(), random_state=42, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks.numpy(), input_ids.numpy(), random_state=42, test_size=0.3)

In [11]:
# Define the model
model = transformers.TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Define the optimizer and the learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)

# Compile the model with SparseCategoricalCrossentropy loss
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train the model
history = model.fit(
    x=(train_inputs, train_masks),
    y=train_labels,
    validation_data=((validation_inputs, validation_masks), validation_labels),
    epochs=2,
    batch_size=32
)

Epoch 1/2
1094/1094 [==============================] - 1098s 953ms/step - loss: 0.3211 - accuracy: 0.8591 - val_loss: 0.2671 - val_accuracy: 0.8859
Epoch 2/2
1094/1094 [==============================] - 1049s 959ms/step - loss: 0.1927 - accuracy: 0.9242 - val_loss: 0.2800 - val_accuracy: 0.8937
